In [1]:
import re
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import nltk
import collections
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.stem.porter import PorterStemmer
from sklearn.cluster import KMeans
from nltk import word_tokenize
from nltk.corpus import stopwords
from sklearn.decomposition import PCA
from sklearn import metrics
!pip install pandas==1.2

In [2]:
nltk.download("stopwords")
nltk.download("wordnet")
nltk.download("reuters")
nltk.download("punkt")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package reuters to /root/nltk_data...
[nltk_data]   Package reuters is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
!pip install pyLDAvis

In [4]:
from nltk.corpus import brown as corpus
nltk.download('brown')

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!


True

In [5]:
for n,item in enumerate(corpus.words(corpus.fileids()[0])[:300]):
    print(item, end=" ")
    if (n%25) ==24:
      print(" ")

The Fulton County Grand Jury said Friday an investigation of Atlanta's recent primary election produced `` no evidence '' that any irregularities took place .  
The jury further said in term-end presentments that the City Executive Committee , which had over-all charge of the election , `` deserves the praise  
and thanks of the City of Atlanta '' for the manner in which the election was conducted . The September-October term jury had been charged  
by Fulton Superior Court Judge Durwood Pye to investigate reports of possible `` irregularities '' in the hard-fought primary which was won by Mayor-nominate Ivan  
Allen Jr. . `` Only a relative handful of such reports was received '' , the jury said , `` considering the widespread interest in  
the election , the number of voters and the size of this city '' . The jury said it did find that many of Georgia's  
registration and election laws `` are outmoded or inadequate and often ambiguous '' . It recommended that Fulton legislators act `

In [6]:
corpus.categories()

['adventure',
 'belles_lettres',
 'editorial',
 'fiction',
 'government',
 'hobbies',
 'humor',
 'learned',
 'lore',
 'mystery',
 'news',
 'religion',
 'reviews',
 'romance',
 'science_fiction']

In [7]:
len(corpus.fileids())

500

In [8]:
docs=[corpus.words(fileid) for fileid in corpus.fileids()]
print(docs[:5])
print("num of docs:", len(docs))

[['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', ...], ['Austin', ',', 'Texas', '--', 'Committee', 'approval', ...], ['Several', 'defendants', 'in', 'the', 'Summerdale', ...], ['Oslo', 'The', 'most', 'positive', 'element', 'to', ...], ['East', 'Providence', 'should', 'organize', 'its', ...]]
num of docs: 500


In [9]:
en_stop = nltk.corpus.stopwords.words('english')
en_stop = ["``","/",",.",".,",";","--",":",")","(",'"','&',"'",'),',',"','-','.,','.,"','.-',"?",">","<","!"]                  \
         +["0","1","2","3","4","5","6","7","8","9","10","11","12","86","1986","1987","000"]                                                      \
         +["said","say","u","v","mln","ct","net","dlrs","tonne","pct","shr","nil","company","lt","share","year","billion","one","price","make","af","would","could","two"]          \
         +en_stop

In [10]:
from nltk.corpus import wordnet as wn 

def preprocess_word(word, stopwordset):
    
    word=word.lower()
    
    if word in [",",".","'","''"]:
        return None
    
    if word in stopwordset:
        return None
    
    lemma = wn.morphy(word)
    if lemma is None:
        return word

    elif lemma in stopwordset: 
        return None
    else:
        return lemma
    
def preprocess_document(document):
    document=[preprocess_word(w, en_stop) for w in document]
    document=[w for w in document if w is not None]
    return document

def preprocess_documents(documents):
    return [preprocess_document(document) for document in documents]

In [11]:
# before
print(docs[0][:25]) 
# after
print(preprocess_documents(docs)[0][:25])

['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', 'Friday', 'an', 'investigation', 'of', "Atlanta's", 'recent', 'primary', 'election', 'produced', '``', 'no', 'evidence', "''", 'that', 'any', 'irregularities', 'took', 'place', '.']
['fulton', 'county', 'grand', 'jury', 'friday', 'investigation', "atlanta's", 'recent', 'primary', 'election', 'produce', 'evidence', 'irregularity', 'take', 'place', 'jury', 'term-end', 'presentment', 'city', 'executive', 'committee', 'over-all', 'charge', 'election', 'deserve']


In [12]:
import gensim
from gensim import corpora

In [13]:
dictionary = corpora.Dictionary(preprocess_documents(docs))
corpus_ = [dictionary.doc2bow(doc) for doc in preprocess_documents(docs)]

In [14]:
print(dictionary.token2id) 

{'$10': 0, '$100': 1, '$3': 2, '$30': 3, '$4': 4, '$50': 5, '1,119': 6, '13': 7, '13th': 8, '18': 9, '1913': 10, '1923': 11, '1937': 12, '1958': 13, '1961': 14, '1962': 15, '29-5': 16, '402': 17, '637': 18, '71': 19, '74': 20, '87-31': 21, 'accept': 22, 'accord': 23, 'achieve': 24, 'act': 25, 'action': 26, 'add': 27, 'additional': 28, 'adjournment': 29, 'adjustment': 30, 'administration': 31, 'administrator': 32, 'afternoon': 33, 'age': 34, 'agree': 35, 'agriculture': 36, 'aid': 37, 'airport': 38, 'aj': 39, 'ala.': 40, 'allen': 41, 'allot': 42, 'allow': 43, 'allowance': 44, 'alpharetta': 45, 'also': 46, 'alternative': 47, 'ambiguous': 48, 'amendment': 49, 'amicable': 50, 'among': 51, 'announce': 52, 'anonymous': 53, 'apparently': 54, 'appoint': 55, 'appointment': 56, 'appraiser': 57, 'approve': 58, 'area': 59, 'arm': 60, 'aside': 61, 'ask': 62, 'asking': 63, 'assistance': 64, 'assistant': 65, 'association': 66, 'atlanta': 67, "atlanta's": 68, 'attend': 69, 'attorney': 70, 'audience': 7

In [15]:
print(corpus_[0][:69]) 

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 2), (26, 4), (27, 3), (28, 2), (29, 2), (30, 1), (31, 2), (32, 1), (33, 1), (34, 2), (35, 1), (36, 1), (37, 1), (38, 2), (39, 1), (40, 1), (41, 2), (42, 1), (43, 2), (44, 1), (45, 1), (46, 3), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 3), (54, 2), (55, 1), (56, 1), (57, 1), (58, 3), (59, 1), (60, 1), (61, 1), (62, 4), (63, 1), (64, 1), (65, 1), (66, 1), (67, 5), (68, 2)]


In [16]:
# before
print([w.lower() for w in corpus.sents(corpus.fileids()[0])[0]])
# after
print(dictionary.doc2bow([w.lower() for w in corpus.sents(corpus.fileids()[0])[0]]))

['the', 'fulton', 'county', 'grand', 'jury', 'said', 'friday', 'an', 'investigation', 'of', "atlanta's", 'recent', 'primary', 'election', 'produced', '``', 'no', 'evidence', "''", 'that', 'any', 'irregularities', 'took', 'place', '.']
[(68, 1), (149, 1), (196, 1), (214, 1), (247, 1), (250, 1), (273, 1), (312, 1), (327, 1), (432, 1), (452, 1), (485, 1)]


In [17]:
ldamodel1 = gensim.models.ldamodel.LdaModel(corpus=corpus_,
                                           num_topics=7,
                                           id2word=dictionary,
                                           alpha=0.1,             
                                           eta=0.1,                  
                                           minimum_probability=0.001    
                                          )

In [18]:
topics = ldamodel1.print_topics(num_words=10)
for topic in topics:
    print(topic)

(0, '0.004*"state" + 0.004*"time" + 0.003*"new" + 0.003*"come" + 0.003*"may" + 0.003*"first" + 0.003*"take" + 0.003*"go" + 0.002*"man" + 0.002*"get"')
(1, '0.004*"get" + 0.004*"like" + 0.004*"use" + 0.003*"time" + 0.003*"man" + 0.003*"new" + 0.003*"know" + 0.003*"may" + 0.002*"go" + 0.002*"come"')
(2, '0.003*"time" + 0.003*"like" + 0.003*"new" + 0.003*"take" + 0.003*"know" + 0.003*"see" + 0.003*"go" + 0.003*"come" + 0.002*"may" + 0.002*"get"')
(3, '0.003*"new" + 0.003*"take" + 0.003*"state" + 0.003*"must" + 0.002*"time" + 0.002*"first" + 0.002*"also" + 0.002*"man" + 0.002*"may" + 0.002*"get"')
(4, '0.004*"new" + 0.003*"state" + 0.003*"first" + 0.003*"may" + 0.003*"come" + 0.002*"know" + 0.002*"time" + 0.002*"use" + 0.002*"many" + 0.002*"even"')
(5, '0.003*"state" + 0.003*"go" + 0.003*"know" + 0.003*"time" + 0.003*"new" + 0.003*"like" + 0.003*"use" + 0.003*"even" + 0.003*"may" + 0.002*"first"')
(6, '0.003*"state" + 0.003*"take" + 0.003*"go" + 0.003*"know" + 0.003*"come" + 0.003*"time" +

In [19]:
for n,item in enumerate(corpus_[0:10]):
    print("document ID "+str(n)+":" ,end="")
    print(ldamodel1.get_document_topics(item))

document ID 0:[(4, 0.9994612)]
document ID 1:[(4, 0.16017233), (5, 0.47234374), (6, 0.36638856)]
document ID 2:[(0, 0.0045346078), (1, 0.0780945), (3, 0.038544253), (4, 0.8408192), (5, 0.03783361)]
document ID 3:[(0, 0.2842234), (2, 0.017443558), (3, 0.22256458), (4, 0.35113263), (5, 0.11962364), (6, 0.004920385)]
document ID 4:[(4, 0.97488075), (5, 0.024649058)]
document ID 5:[(4, 0.0061100894), (5, 0.99330014)]
document ID 6:[(3, 0.005009817), (4, 0.9945482)]
document ID 7:[(0, 0.2599792), (2, 0.0071230675), (4, 0.7304446), (5, 0.0021691152)]
document ID 8:[(4, 0.99947983)]
document ID 9:[(0, 0.20872425), (4, 0.4363529), (5, 0.3545677)]


In [20]:
ldamodel2 = gensim.models.ldamodel.LdaModel(corpus=corpus_,
                                           num_topics=12,
                                           id2word=dictionary,
                                           alpha=0.1,                
                                           eta=0.1,                   
                                           minimum_probability=0.001    
                                          )

In [21]:
topics = ldamodel2.print_topics(num_words=10)
for topic in topics:
    print(topic)

(0, '0.003*"new" + 0.003*"time" + 0.003*"may" + 0.002*"go" + 0.002*"use" + 0.002*"take" + 0.002*"man" + 0.002*"know" + 0.002*"also" + 0.002*"first"')
(1, '0.003*"come" + 0.003*"new" + 0.003*"go" + 0.003*"time" + 0.003*"take" + 0.003*"many" + 0.003*"may" + 0.002*"first" + 0.002*"like" + 0.002*"use"')
(2, '0.003*"take" + 0.003*"get" + 0.003*"state" + 0.003*"new" + 0.003*"man" + 0.003*"go" + 0.003*"time" + 0.003*"know" + 0.003*"come" + 0.002*"first"')
(3, '0.004*"go" + 0.003*"come" + 0.003*"new" + 0.003*"time" + 0.003*"get" + 0.003*"know" + 0.003*"take" + 0.002*"even" + 0.002*"first" + 0.002*"see"')
(4, '0.004*"like" + 0.003*"know" + 0.003*"state" + 0.003*"time" + 0.003*"may" + 0.003*"take" + 0.002*"get" + 0.002*"go" + 0.002*"new" + 0.002*"first"')
(5, '0.003*"new" + 0.003*"time" + 0.003*"may" + 0.003*"know" + 0.003*"come" + 0.003*"first" + 0.003*"state" + 0.003*"take" + 0.003*"go" + 0.003*"man"')
(6, '0.003*"new" + 0.003*"may" + 0.003*"like" + 0.003*"time" + 0.003*"take" + 0.003*"first" 

In [22]:
for n,item in enumerate(corpus_[0:10]):
    print("document ID "+str(n)+":" ,end="")
    print(ldamodel2.get_document_topics(item))

document ID 0:[(5, 0.28836295), (7, 0.008846485), (11, 0.70198256)]
document ID 1:[(1, 0.001463685), (5, 0.3618517), (7, 0.022653949), (8, 0.0014143427), (11, 0.6120246)]
document ID 2:[(1, 0.25401703), (5, 0.014038796), (7, 0.6651587), (11, 0.06609039)]
document ID 3:[(1, 0.039362628), (2, 0.2905167), (5, 0.32061017), (7, 0.30811384), (8, 0.03563692), (11, 0.004603306)]
document ID 4:[(4, 0.37438068), (5, 0.029358989), (7, 0.0200679), (8, 0.05353828), (11, 0.5217254)]
document ID 5:[(0, 0.0025569994), (2, 0.36402503), (5, 0.24964732), (7, 0.33036348), (9, 0.05276612)]
document ID 6:[(1, 0.24199751), (5, 0.2422133), (10, 0.5147403)]
document ID 7:[(5, 0.082230575), (7, 0.91682327)]
document ID 8:[(0, 0.060056373), (1, 0.06260639), (5, 0.021528665), (6, 0.44024786), (7, 0.32722944), (11, 0.08781129)]
document ID 9:[(1, 0.04954221), (2, 0.003017473), (4, 0.0064027533), (5, 0.28426242), (6, 0.055546343), (7, 0.22335231), (10, 0.26254573), (11, 0.11453973)]


In [23]:
categories = [corpus.categories(fileid) for fileid in corpus.fileids()]
print(categories)

[['news'], ['news'], ['news'], ['news'], ['news'], ['news'], ['news'], ['news'], ['news'], ['news'], ['news'], ['news'], ['news'], ['news'], ['news'], ['news'], ['news'], ['news'], ['news'], ['news'], ['news'], ['news'], ['news'], ['news'], ['news'], ['news'], ['news'], ['news'], ['news'], ['news'], ['news'], ['news'], ['news'], ['news'], ['news'], ['news'], ['news'], ['news'], ['news'], ['news'], ['news'], ['news'], ['news'], ['news'], ['editorial'], ['editorial'], ['editorial'], ['editorial'], ['editorial'], ['editorial'], ['editorial'], ['editorial'], ['editorial'], ['editorial'], ['editorial'], ['editorial'], ['editorial'], ['editorial'], ['editorial'], ['editorial'], ['editorial'], ['editorial'], ['editorial'], ['editorial'], ['editorial'], ['editorial'], ['editorial'], ['editorial'], ['editorial'], ['editorial'], ['editorial'], ['reviews'], ['reviews'], ['reviews'], ['reviews'], ['reviews'], ['reviews'], ['reviews'], ['reviews'], ['reviews'], ['reviews'], ['reviews'], ['reviews']

In [24]:
n=1
# nth document's topic distribution
print(ldamodel1.get_document_topics(corpus_[n]))
# nth document's category
print(categories[n])
# show the original document
print(" ".join(docs[n]))

[(3, 0.0016325754), (4, 0.1757412), (5, 0.45933717), (6, 0.36303535)]
['news']
Austin , Texas -- Committee approval of Gov. Price Daniel's `` abandoned property '' act seemed certain Thursday despite the adamant protests of Texas bankers . Daniel personally led the fight for the measure , which he had watered down considerably since its rejection by two previous Legislatures , in a public hearing before the House Committee on Revenue and Taxation . Under committee rules , it went automatically to a subcommittee for one week . But questions with which committee members taunted bankers appearing as witnesses left little doubt that they will recommend passage of it . Daniel termed `` extremely conservative '' his estimate that it would produce 17 million dollars to help erase an anticipated deficit of 63 million dollars at the end of the current fiscal year next Aug. 31 . He told the committee the measure would merely provide means of enforcing the escheat law which has been on the books 

In [28]:
n=1
# nth document's topic distribution
print(ldamodel2.get_document_topics(corpus_[n]))
# nth document's category
print(categories[n])
# show the original document
print(" ".join(docs[n]))

[(1, 0.0020793304), (5, 0.35658062), (7, 0.01637351), (11, 0.62348473)]
['news']
Austin , Texas -- Committee approval of Gov. Price Daniel's `` abandoned property '' act seemed certain Thursday despite the adamant protests of Texas bankers . Daniel personally led the fight for the measure , which he had watered down considerably since its rejection by two previous Legislatures , in a public hearing before the House Committee on Revenue and Taxation . Under committee rules , it went automatically to a subcommittee for one week . But questions with which committee members taunted bankers appearing as witnesses left little doubt that they will recommend passage of it . Daniel termed `` extremely conservative '' his estimate that it would produce 17 million dollars to help erase an anticipated deficit of 63 million dollars at the end of the current fiscal year next Aug. 31 . He told the committee the measure would merely provide means of enforcing the escheat law which has been on the book

In [25]:
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


In [26]:
lda_display = pyLDAvis.gensim_models.prepare(ldamodel1, corpus_, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

In [27]:
lda_display = pyLDAvis.gensim_models.prepare(ldamodel2, corpus_, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)